In [2]:
import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import re

In [3]:
def make_soup(url):
#     r = requests.get(url)
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    r = urlopen(req).read()
    soup = BeautifulSoup(r, "html")
    return soup

# put urls in a list
def get_xml_urls(soup):
    urls = [loc.string for loc in soup.find_all("loc")]
    return urls

# get media urls
def get_urls(xml):
    list_of_urls = []
    soup = make_soup(xml)
    urls = get_xml_urls(soup)
    # loop through the urls
    for url in urls:
        list_of_urls.append(url)
    return list_of_urls

In [4]:
metareel_sitemap = "https://www.metareel.com/sitemaps/show_sitemap_google.xml"
urls = get_urls(metareel_sitemap)

In [5]:
len(urls)

3173

In [33]:

content_list = []
for idx, url in enumerate(urls[1913:2001]):
    content_dict = {}
    soup = make_soup(url)
    print("fetching",1913+idx)
    try:
        movie_name = soup.find("h1","details-title").text
        content_dict['show_name'] = movie_name
    except:
        content_dict['show_name'] = ''
    try:
        details = soup.find("div","detail-filter pb-1")
        content_dict['type'] = details.find("span").text
        content_dict['type'] = "TV Show"
        genre = []
        for a in details.find_all('a'):
            genre.append(a.text)
        content_dict['genre'] = genre
    except:
        content_dict['type'] = "TV Show"
        content_dict['genre'] = []
    try:
        language_tag = a.find_next("span").find_next("span")
        content_dict['language'] = language_tag.text
    except:
        content_dict['language'] = ""
    try:
        language = a.find_next("span").find_next("span")
        content_dict['release_date'] = language.find_next("span").find_next("span").text.replace("Release Date : ","")
    except:
        content_dict['release_date'] = ""
    try:
        content_dict['imdb_rating'] = soup.find("div","movie-rating-div").text.replace("/10","")
    except:
        content_dict['imdb_rating'] = ""
 
    try:
        stream = []
        
        watch = soup.find("div", "card-component padd")
        tag = watch.find("div","panel-chnl-wrp")
        for link in tag.findAll('a'):
            stream.append(link['href'])
      
    except:
        stream = []
    
    content_dict['stream url'] = stream
    
    try:
        content_dict['cast'] = [tag.text for tag in soup.find("div",attrs={"name":"Cast"}).find_all('a') if tag.text]
    except:
        content_dict['cast'] = []
    try:
        content_dict['crew'] = [tag.text for tag in soup.find("div",attrs={"name":"Crew"}).find_all('a') if tag.text]
    except:
        content_dict['crew'] = []
    content_list.append(content_dict)

fetching 1913
fetching 1914
fetching 1915
fetching 1916
fetching 1917
fetching 1918
fetching 1919
fetching 1920
fetching 1921
fetching 1922
fetching 1923
fetching 1924
fetching 1925
fetching 1926
fetching 1927
fetching 1928
fetching 1929
fetching 1930
fetching 1931
fetching 1932
fetching 1933
fetching 1934
fetching 1935
fetching 1936
fetching 1937
fetching 1938
fetching 1939
fetching 1940
fetching 1941
fetching 1942
fetching 1943
fetching 1944
fetching 1945
fetching 1946
fetching 1947
fetching 1948
fetching 1949
fetching 1950
fetching 1951
fetching 1952
fetching 1953
fetching 1954
fetching 1955
fetching 1956
fetching 1957
fetching 1958
fetching 1959
fetching 1960
fetching 1961
fetching 1962
fetching 1963
fetching 1964
fetching 1965
fetching 1966
fetching 1967
fetching 1968
fetching 1969
fetching 1970
fetching 1971
fetching 1972
fetching 1973
fetching 1974
fetching 1975
fetching 1976
fetching 1977
fetching 1978
fetching 1979
fetching 1980
fetching 1981
fetching 1982
fetching 1983
fetchi

In [34]:

movies_df = pd.DataFrame.from_dict(content_list, orient='columns')
movies_df

,show_name,type,genre,language,release_date,imdb_rating,stream url,cast,crew
0,Deadly Secrets,TV Show,[Crime],English,2019-1-1,,[],[Alain Goulem],"[Hans Rosenstein, Debbie Travis, Scott Bailey,..."
1,Killing Time,TV Show,[],English,2019-1-1,,[],[Mocean Melvin],"[Kelly McPherson, Emre Sahin, Sarah Wetherbee,..."
2,The Twinstitute,TV Show,[Documentary],English,2019-1-2,,[],"[Chris van Tulleken, Xand van Tulleken]","[Leanne Klein, Emily Shields, Ed Watkins, Ed W..."
3,Power Rangers Beast Morphers,TV Show,"[Action, Adventure, Drama]",English,2019-2-2,,[https://www.netflix.com/in/title/81192594],"[Jacqueline Scislowski, Abraham Rodriguez, Col...","[Oliver Driver, Riccardo Pellizzeri, Becca Bar..."
4,Tom Kerridge's Fresh Start,TV Show,[],English,2019-2-2,,[],[Tom Kerridge],"[Richard Bowron, Sarah Myland, Sarah Myland]"
...,...,...,...,...,...,...,...,...,...
83,La Voz (us),TV Show,[],English,2018-8-3,,[],"[Jorge Bernal, Luis Fonsi, Alejandra Guzmán, W...",[]
84,American Princess [ 2019 ],TV Show,[Drama],English,2019-6-2,,[],"[Georgia Flood, Mary Hollis Inboden, Seana Kof...",[]
85,Fast Layne [ 2019 ],TV Show,[Comedy],English,2019-2-15,,[],"[Sophie Pollono, Sofia Rosinsky, Brandon Rosse...","[Tom Burkhard, Matt Dearborn, Travis Braun]"
86,The Twilight Zone [ 2019 ],TV Show,"[Mystery, Drama]",English,2019-3-31,5.7,[https://www.voot.com/shows/the-twilight-zone/...,"[Jordan Peele, David Epstein, Kelly Ann Woods,...","[Ana Lily Amirpour, Christina Choe, Mathias He..."


In [35]:
movies_df.columns

Index(['show_name', 'type', 'genre', 'language', 'release_date', 'imdb_rating',
       'stream url', 'cast', 'crew'],
      dtype='object')

In [36]:
movies_df.to_csv('new.csv', mode = 'a')